# Data Integration & EDA
## Power Outage Prediction Analysis

This notebook performs data integration and exploratory data analysis on cleaned weather and outage datasets.

**Input**: Cleaned weather and outage data from data loading notebook
**Output**: Integrated dataset and statistical analysis ready for modeling

**Sections**:
- Data loading and integration 
- Geographic reporting bias analysis
- Exploratory data analysis
- Statistical summaries and correlations

## Load Cleaned Data

First, load the cleaned datasets from the data loading notebook.

In [10]:
# Load the cleaned datasets
import pandas as pd
from pathlib import Path

# Load data helper function
def load_cleaned_data():
    """Load cleaned weather and outage datasets from CSV files"""
    data_dir = Path("C:/Users/Ani/my_projects/AI-Studio-Project/datasets/data")
    weather_file = data_dir / "weather_cleaned.csv"
    outage_file = data_dir / "outages_cleaned.csv"
    
    if weather_file.exists() and outage_file.exists():
        weather_df = pd.read_csv(weather_file)
        outage_df = pd.read_csv(outage_file)
        
        # Convert date columns back to datetime
        weather_df['date'] = pd.to_datetime(weather_df['date'])
        outage_df['date'] = pd.to_datetime(outage_df['date'])
        outage_df['run_start_time'] = pd.to_datetime(outage_df['run_start_time'])
        
        return weather_df, outage_df
    else:
        raise FileNotFoundError("No cleaned data files found - run data cleaning first")

# Load the data
weather_pivot, outage_clean = load_cleaned_data()

print("Data loaded successfully")
print(f"Weather data: {weather_pivot.shape}")
print(f"Outage data: {outage_clean.shape}")

Data loaded successfully
Weather data: (7944599, 6)
Outage data: (3993154, 7)


In [11]:
weather_pivot.head()

,fips_code,county_name,date,prcp,tmax,tmin
0,1001,AL: Autauga,2014-01-01,0.00,7.92,3.12
1,1001,AL: Autauga,2014-01-02,7.54,9.14,4.07
2,1001,AL: Autauga,2014-01-03,1.41,12.62,-4.65
3,1001,AL: Autauga,2014-01-04,0.00,4.46,-5.44
4,1001,AL: Autauga,2014-01-05,0.00,9.72,-4.71


In [12]:
outage_clean.head()

,fips_code,county,state,customers_out,run_start_time,date,outage_occurred
0,1037,Coosa,Alabama,12.0,2014-11-01 04:00:00,2014-11-01,1
1,1051,Elmore,Alabama,7.0,2014-11-01 04:00:00,2014-11-01,1
2,1109,Pike,Alabama,1.0,2014-11-01 04:00:00,2014-11-01,1
3,1121,Talladega,Alabama,31.0,2014-11-01 04:00:00,2014-11-01,1
4,4017,Navajo,Arizona,1.0,2014-11-01 04:00:00,2014-11-01,1


## Data Integration

Merge weather and outage datasets, creating a complete analysis-ready dataset.

**Integration Strategy**
- Left join weather data (complete coverage) with outage data (partial coverage)
- Filter to counties with verified outage reporting to avoid bias
- Clean geographic identifiers for consistent analysis

**Key Processing Steps**
1. Merge datasets on fips_code and date
2. Fill missing outage indicators (NaN = no outage)
3. Restrict to counties with demonstrated reporting capability
4. Extract clean geographic names from weather data

In [13]:
# Data Integration - Merge weather and outage datasets
print("Integrating weather and outage datasets...")

# Merge weather and outage data (left join to preserve all weather observations)
merged_df = weather_pivot.merge(
    outage_clean[['fips_code', 'date', 'outage_occurred', 'customers_out', 'county', 'state', 'run_start_time']], 
    on=['fips_code', 'date'], 
    how='left'
)

# Fill NaN values for outage indicators (NaN = no outage occurred)
merged_df['outage_occurred'] = merged_df['outage_occurred'].fillna(0).astype(int)
merged_df['customers_out'] = merged_df['customers_out'].fillna(0).astype(int)

# Filter to counties with outage reporting systems to avoid statistical bias
counties_with_reporting = set(outage_clean['fips_code'].unique())
merged_df = merged_df[merged_df['fips_code'].isin(counties_with_reporting)].copy()

print(f"Final dataset: {merged_df.shape}")
print(f"Counties included: {merged_df['fips_code'].nunique():,}")
print(f"Outage rate: {merged_df['outage_occurred'].mean():.1%}")
print(f"Date range: {merged_df['date'].min()} to {merged_df['date'].max()}")
print(f"Columns: {list(merged_df.columns)}")


Integrating weather and outage datasets...
Final dataset: (3976135, 11)
Counties included: 1,555
Outage rate: 49.9%
Date range: 2014-01-01 00:00:00 to 2020-12-31 00:00:00
Columns: ['fips_code', 'county_name', 'date', 'prcp', 'tmax', 'tmin', 'outage_occurred', 'customers_out', 'county', 'state', 'run_start_time']


## Geographic Data Cleaning

Clean and standardize geographic identifiers from weather data source.

In [14]:
# Extract clean geographic data from weather dataset

# Check if cleaning is already done
if 'county_name' not in merged_df.columns:
    print("Geographic cleaning already completed.")
    print(f"Dataset: {merged_df.shape[0]:,} observations across {merged_df['state'].nunique()} states, {merged_df['county'].nunique()} counties")
else:
    print("Extracting clean geographic data...")
    
    # State abbreviation mapping
    state_map = {
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
        'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
        'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
        'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
        'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
        'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
        'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
        'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
        'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
        'DC': 'District of Columbia'
    }

    # Extract from county_name format: "ST: County Name"
    merged_df['state'] = merged_df['county_name'].str.split(':').str[0].map(state_map)
    merged_df['county'] = merged_df['county_name'].str.split(':').str[1].str.strip()
    
    # Clean up
    merged_df = merged_df.drop('county_name', axis=1)
    print("Geographic cleaning completed!")

# Final dataset summary
print(f"\nIntegrated Dataset Summary:")
print(f"Shape: {merged_df.shape}")
print(f"Coverage: {merged_df['state'].nunique()} states, {merged_df['county'].nunique()} counties")
print(f"Outage rate: {merged_df['outage_occurred'].mean():.1%}")

merged_df.head(10)

Extracting clean geographic data...
Geographic cleaning completed!

Integrated Dataset Summary:
Shape: (3976135, 10)
Coverage: 43 states, 1066 counties
Outage rate: 49.9%


,fips_code,date,prcp,tmax,tmin,outage_occurred,customers_out,county,state,run_start_time
0,1001,2014-01-01,0.00,7.92,3.12,0,0,Autauga,Alabama,NaT
1,1001,2014-01-02,7.54,9.14,4.07,0,0,Autauga,Alabama,NaT
2,1001,2014-01-03,1.41,12.62,-4.65,0,0,Autauga,Alabama,NaT
3,1001,2014-01-04,0.00,4.46,-5.44,0,0,Autauga,Alabama,NaT
4,1001,2014-01-05,0.00,9.72,-4.71,0,0,Autauga,Alabama,NaT
5,1001,2014-01-06,2.39,15.71,-3.26,0,0,Autauga,Alabama,NaT
6,1001,2014-01-07,0.00,-0.88,-11.17,0,0,Autauga,Alabama,NaT
7,1001,2014-01-08,0.00,-1.97,-11.40,0,0,Autauga,Alabama,NaT
8,1001,2014-01-09,0.00,6.69,-8.77,0,0,Autauga,Alabama,NaT
9,1001,2014-01-10,3.90,10.84,-1.10,0,0,Autauga,Alabama,NaT


## Temporal Feature Engineering

Extract meaningful temporal components for predictive modeling.

**Rationale for Temporal Decomposition**

Raw dates are poor features for machine learning models. Decomposing dates into components allows models to:
- **Learn seasonal patterns**: Winter vs summer outage rates
- **Detect weekly cycles**: Weekend vs weekday maintenance patterns  
- **Identify peak hours**: Time-of-day when outages are most likely
- **Capture trends**: Multi-year infrastructure changes

**Feature Extraction Strategy**
- From `date`: Month, day of week, season, year (for all observations)
- From `run_start_time`: Hour of day (only for outage events - when timing matters)
- Preserve original `date` for sorting and potential date-based joins
- Keep `run_start_time` for detailed temporal analysis of outage events

In [15]:
# Temporal feature engineering

print("Extracting temporal features...")

# Extract date components (for all observations)
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day_of_week'] = merged_df['date'].dt.dayofweek  # 0=Monday, 6=Sunday
merged_df['day_name'] = merged_df['date'].dt.day_name()

# Create season feature
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

merged_df['season'] = merged_df['month'].apply(get_season)

# Extract hour from run_start_time (only for outage events)
merged_df['outage_hour'] = merged_df['run_start_time'].dt.hour

print("Temporal features extracted!")

# Show summary of new features
print(f"\nTemporal Feature Summary:")
print(f"Years covered: {merged_df['year'].min()} - {merged_df['year'].max()}")
print(f"Seasons: {merged_df['season'].value_counts().to_dict()}")
print(f"Day of week distribution: {merged_df['day_name'].value_counts().to_dict()}")

# Show outage hour distribution (only for actual outages)
outage_hours = merged_df[merged_df['outage_occurred'] == 1]['outage_hour'].value_counts().sort_index()
print(f"\nOutage hour distribution (top 5):")
print(outage_hours.head())

print(f"\nDataset shape after temporal features: {merged_df.shape}")
print(f"New columns: {list(merged_df.columns)}")

Extracting temporal features...
Temporal features extracted!

Temporal Feature Summary:
Years covered: 2014 - 2020
Seasons: {'Spring': 1001420, 'Summer': 1001420, 'Fall': 990535, 'Winter': 982760}
Day of week distribution: {'Wednesday': 569130, 'Thursday': 569130, 'Friday': 567575, 'Saturday': 567575, 'Sunday': 567575, 'Monday': 567575, 'Tuesday': 567575}

Outage hour distribution (top 5):
outage_hour
0.0    768184
1.0    102560
2.0     69734
3.0     52300
4.0     45121
Name: count, dtype: int64

Dataset shape after temporal features: (3976135, 16)
New columns: ['fips_code', 'date', 'prcp', 'tmax', 'tmin', 'outage_occurred', 'customers_out', 'county', 'state', 'run_start_time', 'year', 'month', 'day_of_week', 'day_name', 'season', 'outage_hour']


In [16]:
# Sample observations with new temporal features

print("Sample observations showing temporal features:")
print("=" * 50)

# Show outage events with temporal details
print("\nOutage events with temporal context:")
outage_sample = merged_df[merged_df['outage_occurred'] == 1].head(5)
display(outage_sample[['county', 'state', 'date', 'year', 'month', 'season', 'day_name', 
                      'outage_hour', 'customers_out', 'run_start_time']])

# Show no-outage events 
print("\nNo-outage events with temporal context:")
no_outage_sample = merged_df[merged_df['outage_occurred'] == 0].head(5)
display(no_outage_sample[['county', 'state', 'date', 'year', 'month', 'season', 'day_name', 
                         'outage_hour', 'customers_out', 'run_start_time']])

Sample observations showing temporal features:

Outage events with temporal context:


,county,state,date,year,month,season,day_name,outage_hour,customers_out,run_start_time
304,Autauga,Alabama,2014-11-01,2014,11,Fall,Saturday,16.0,3,2014-11-01 16:15:00
305,Autauga,Alabama,2014-11-02,2014,11,Fall,Sunday,18.0,42,2014-11-02 18:45:00
306,Autauga,Alabama,2014-11-03,2014,11,Fall,Monday,14.0,5,2014-11-03 14:00:00
309,Autauga,Alabama,2014-11-06,2014,11,Fall,Thursday,19.0,2,2014-11-06 19:45:00
310,Autauga,Alabama,2014-11-07,2014,11,Fall,Friday,1.0,2,2014-11-07 01:00:00



No-outage events with temporal context:


,county,state,date,year,month,season,day_name,outage_hour,customers_out,run_start_time
0,Autauga,Alabama,2014-01-01,2014,1,Winter,Wednesday,NaN,0,NaT
1,Autauga,Alabama,2014-01-02,2014,1,Winter,Thursday,NaN,0,NaT
2,Autauga,Alabama,2014-01-03,2014,1,Winter,Friday,NaN,0,NaT
3,Autauga,Alabama,2014-01-04,2014,1,Winter,Saturday,NaN,0,NaT
4,Autauga,Alabama,2014-01-05,2014,1,Winter,Sunday,NaN,0,NaT


## Exploratory Data Analysis for Predictive Modeling

**Objective**: Analyze patterns and relationships to support **≥85% accuracy** outage prediction model.

**Key Analysis Areas**:
1. **Target Variable Analysis**: Class balance and temporal patterns
2. **Weather Threshold Analysis**: Critical thresholds for outage prediction  
3. **Temporal Patterns**: Seasonal, monthly, and hourly outage risks
4. **Geographic Risk Assessment**: High-risk regions and baseline rates
5. **Feature Relationships**: Correlations and interaction effects
6. **Baseline Model Insights**: Simple rules for comparison

In [17]:
# 1. Dataset Overview and Target Analysis

print("POWER OUTAGE PREDICTION - EXPLORATORY DATA ANALYSIS")
print("="*60)

# Basic dataset characteristics
print(f"\nDataset Overview:")
print(f"Total observations: {len(merged_df):,}")
print(f"Variables: {len(merged_df.columns)}")
print(f"Time span: {(merged_df['date'].max() - merged_df['date'].min()).days} days ({merged_df['year'].min()}-{merged_df['year'].max()})")
print(f"Geographic coverage: {merged_df['fips_code'].nunique()} counties across {merged_df['state'].nunique()} states")

# Target variable analysis
outage_counts = merged_df['outage_occurred'].value_counts()
outage_rate = merged_df['outage_occurred'].mean()

print(f"\nTarget Variable Analysis:")
print(f"Overall outage rate: {outage_rate:.1%}")
print(f"No outage (0): {outage_counts[0]:,} observations ({outage_counts[0]/len(merged_df):.1%})")
print(f"Outage (1): {outage_counts[1]:,} observations ({outage_counts[1]/len(merged_df):.1%})")

POWER OUTAGE PREDICTION - EXPLORATORY DATA ANALYSIS

Dataset Overview:
Total observations: 3,976,135
Variables: 16
Time span: 2556 days (2014-2020)
Geographic coverage: 1555 counties across 43 states

Target Variable Analysis:
Overall outage rate: 49.9%
No outage (0): 1,991,833 observations (50.1%)
Outage (1): 1,984,302 observations (49.9%)


In [18]:
# 2. Weather Threshold Analysis

print("\n" + "="*50)
print("WEATHER THRESHOLD ANALYSIS")
print("="*50)

# Find weather thresholds where outage rates spike
def analyze_weather_thresholds(var_name, thresholds):
    print(f"\n{var_name.upper()} Threshold Analysis:")
    var_data = merged_df[var_name]
    
    for threshold in thresholds:
        if 'min' in var_name:
            condition = var_data <= threshold
            operator = "≤"
        else:
            condition = var_data >= threshold  
            operator = "≥"
            
        subset = merged_df[condition]
        if len(subset) > 100:  # Only analyze if sufficient data
            outage_rate = subset['outage_occurred'].mean()
            baseline_rate = merged_df['outage_occurred'].mean()
            risk_ratio = outage_rate / baseline_rate
            
            print(f"  {var_name} {operator} {threshold}: {outage_rate:.1%} outage rate "
                  f"({len(subset):,} observations, {risk_ratio:.1f}x baseline)")

# Temperature thresholds
analyze_weather_thresholds('tmin', [-20, -15, -10, -5, 0])
analyze_weather_thresholds('tmax', [30, 35, 40, 45])

# Precipitation thresholds  
analyze_weather_thresholds('prcp', [10, 20, 25, 30, 50, 75])

print(f"\nWeather variable summary statistics:")
weather_stats = merged_df[['tmax', 'tmin', 'prcp']].describe()
display(weather_stats)


WEATHER THRESHOLD ANALYSIS

TMIN Threshold Analysis:
  tmin ≤ -20: 40.3% outage rate (65,928 observations, 0.8x baseline)
  tmin ≤ -15: 43.5% outage rate (147,852 observations, 0.9x baseline)
  tmin ≤ -10: 46.6% outage rate (298,129 observations, 0.9x baseline)
  tmin ≤ -5: 49.1% outage rate (593,503 observations, 1.0x baseline)
  tmin ≤ 0: 50.2% outage rate (1,170,011 observations, 1.0x baseline)

TMAX Threshold Analysis:
  tmax ≥ 30: 52.2% outage rate (617,315 observations, 1.0x baseline)
  tmax ≥ 35: 53.8% outage rate (62,203 observations, 1.1x baseline)
  tmax ≥ 40: 55.0% outage rate (1,475 observations, 1.1x baseline)

PRCP Threshold Analysis:
  prcp ≥ 10: 50.1% outage rate (367,977 observations, 1.0x baseline)
  prcp ≥ 20: 50.3% outage rate (146,750 observations, 1.0x baseline)
  prcp ≥ 25: 50.1% outage rate (96,368 observations, 1.0x baseline)
  prcp ≥ 30: 49.7% outage rate (64,553 observations, 1.0x baseline)
  prcp ≥ 50: 48.8% outage rate (16,061 observations, 1.0x baseline)


,tmax,tmin,prcp
count,3.976135e+06,3.976135e+06,3.976135e+06
mean,1.798884e+01,6.021172e+00,2.933239e+00
std,1.157682e+01,1.074748e+01,7.655854e+00
min,-3.086000e+01,-4.262000e+01,0.000000e+00
25%,9.550000e+00,-1.290000e+00,0.000000e+00
50%,2.004000e+01,6.650000e+00,0.000000e+00
75%,2.756000e+01,1.470000e+01,2.020000e+00
max,4.857000e+01,3.211000e+01,3.308000e+02


In [19]:
# 3. Temporal Pattern Analysis

print("\n" + "="*50)
print("TEMPORAL PATTERN ANALYSIS")
print("="*50)

# Monthly patterns
print("\nMonthly Outage Patterns:")
monthly_stats = merged_df.groupby('month')['outage_occurred'].agg(['count', 'mean']).round(3)
monthly_stats.columns = ['observations', 'outage_rate']
monthly_stats = monthly_stats.reindex(['January', 'February', 'March', 'April', 'May', 'June',
                                     'July', 'August', 'September', 'October', 'November', 'December'])
display(monthly_stats)

# Seasonal patterns
print("\nSeasonal Outage Patterns:")
seasonal_stats = merged_df.groupby('season')['outage_occurred'].agg(['count', 'mean']).round(3)
seasonal_stats.columns = ['observations', 'outage_rate']
display(seasonal_stats)

# Day of week patterns
print("\nDay of Week Patterns:")
dow_stats = merged_df.groupby('day_of_week')['outage_occurred'].agg(['count', 'mean']).round(3)
dow_stats.columns = ['observations', 'outage_rate']
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_stats = dow_stats.reindex(dow_order)
display(dow_stats)

# Hourly patterns (if available)
if 'outage_hour' in merged_df.columns:
    print("\nHourly Outage Patterns:")
    hourly_stats = merged_df.groupby('outage_hour')['outage_occurred'].agg(['count', 'mean']).round(3)
    hourly_stats.columns = ['observations', 'outage_rate']
    print(f"Peak hour: {hourly_stats['outage_rate'].idxmax()}:00 with {hourly_stats['outage_rate'].max():.1%} rate")
    print(f"Low hour: {hourly_stats['outage_rate'].idxmin()}:00 with {hourly_stats['outage_rate'].min():.1%} rate")

# Year over year trends
print("\nYear-over-Year Trends:")
yearly_stats = merged_df.groupby('year')['outage_occurred'].agg(['count', 'mean']).round(3)
yearly_stats.columns = ['observations', 'outage_rate']
display(yearly_stats)


TEMPORAL PATTERN ANALYSIS

Monthly Outage Patterns:


,observations,outage_rate
month,,
January,NaN,NaN
February,NaN,NaN
March,NaN,NaN
April,NaN,NaN
May,NaN,NaN
June,NaN,NaN
July,NaN,NaN
August,NaN,NaN
September,NaN,NaN



Seasonal Outage Patterns:


,observations,outage_rate
season,,
Fall,990535,0.517
Spring,1001420,0.479
Summer,1001420,0.529
Winter,982760,0.470



Day of Week Patterns:


,observations,outage_rate
day_of_week,,
Monday,NaN,NaN
Tuesday,NaN,NaN
Wednesday,NaN,NaN
Thursday,NaN,NaN
Friday,NaN,NaN
Saturday,NaN,NaN
Sunday,NaN,NaN



Hourly Outage Patterns:
Peak hour: 0.0:00 with 100.0% rate
Low hour: 0.0:00 with 100.0% rate

Year-over-Year Trends:


,observations,outage_rate
year,,
2014,567575,0.061
2015,567575,0.420
2016,569130,0.445
2017,567575,0.497
2018,567575,0.633
2019,567575,0.682
2020,569130,0.754


In [20]:
# 4. Geographic Risk Assessment

print("\n" + "="*50)
print("GEOGRAPHIC RISK ASSESSMENT")
print("="*50)

# State-level analysis
print("\nTop 10 States by Outage Rate:")
state_stats = merged_df.groupby('state')['outage_occurred'].agg(['count', 'mean']).round(3)
state_stats.columns = ['observations', 'outage_rate']
state_stats = state_stats[state_stats['observations'] >= 1000]  # Minimum sample size
top_states = state_stats.sort_values('outage_rate', ascending=False).head(10)
display(top_states)

print("\nBottom 10 States by Outage Rate:")
bottom_states = state_stats.sort_values('outage_rate', ascending=True).head(10)
display(bottom_states)

# County-level analysis  
print("\nTop 10 Counties by Outage Rate (min 500 observations):")
county_stats = merged_df.groupby(['state', 'county'])['outage_occurred'].agg(['count', 'mean']).round(3)
county_stats.columns = ['observations', 'outage_rate']
county_stats = county_stats[county_stats['observations'] >= 500]
top_counties = county_stats.sort_values('outage_rate', ascending=False).head(10)
display(top_counties)

# Geographic coverage summary
print(f"\nGeographic Coverage Summary:")
print(f"  States: {merged_df['state'].nunique()}")
print(f"  Counties: {merged_df['county'].nunique()}")
print(f"  FIPS codes: {merged_df['fips_code'].nunique()}")

# Regional patterns
if 'state' in merged_df.columns:
    # Define regions
    regions = {
        'Northeast': ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 
                     'New York', 'New Jersey', 'Pennsylvania'],
        'Southeast': ['Delaware', 'Maryland', 'Virginia', 'West Virginia', 'Kentucky', 'Tennessee',
                     'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Alabama', 'Mississippi', 'Arkansas', 'Louisiana'],
        'Midwest': ['Ohio', 'Indiana', 'Illinois', 'Michigan', 'Wisconsin', 'Minnesota', 'Iowa', 'Missouri',
                   'North Dakota', 'South Dakota', 'Nebraska', 'Kansas'],
        'Southwest': ['Texas', 'Oklahoma', 'New Mexico', 'Arizona'],
        'West': ['Colorado', 'Wyoming', 'Montana', 'Idaho', 'Washington', 'Oregon', 'Utah', 'Nevada', 'California',
                'Alaska', 'Hawaii']
    }
    
    # Create region mapping
    state_to_region = {}
    for region, states in regions.items():
        for state in states:
            state_to_region[state] = region
    
    merged_df['region'] = merged_df['state'].map(state_to_region)
    
    print("\nRegional Outage Patterns:")
    regional_stats = merged_df.groupby('region')['outage_occurred'].agg(['count', 'mean']).round(3)
    regional_stats.columns = ['observations', 'outage_rate']
    display(regional_stats)


GEOGRAPHIC RISK ASSESSMENT

Top 10 States by Outage Rate:


,observations,outage_rate
state,,
Illinois,2557,0.876
Idaho,7671,0.861
Oklahoma,53697,0.842
Indiana,168762,0.800
Georgia,20456,0.778
Utah,74153,0.769
Montana,58811,0.765
Nebraska,35798,0.759
South Dakota,166205,0.741



Bottom 10 States by Outage Rate:


,observations,outage_rate
state,,
New York,132964,0.146
North Carolina,202003,0.170
South Carolina,117622,0.201
West Virginia,138078,0.214
Kentucky,7671,0.254
Louisiana,112508,0.315
Florida,155977,0.320
Wisconsin,163648,0.336
Connecticut,20456,0.339



Top 10 Counties by Outage Rate (min 500 observations):


observations  outage_rate
state        county                                      
Wisconsin    Florence County            2557        0.881
Pennsylvania Lycoming County            2557        0.881
South Dakota Davison County             2557        0.881
             Faulk County               2557        0.881
Minnesota    Otter Tail County          2557        0.881
Maine        York County                2557        0.881
Iowa         Monroe County              2557        0.881
Oklahoma     Alfalfa County             2557        0.881
Pennsylvania Elk County                 2557        0.881
Indiana      Monroe County              2557        0.881


Geographic Coverage Summary:
  States: 43
  Counties: 1066
  FIPS codes: 1555

Regional Outage Patterns:


,observations,outage_rate
region,,
Midwest,1311741,0.569
Northeast,480716,0.468
Southeast,1355210,0.394
Southwest,222459,0.525
West,606009,0.598


In [21]:
# Save the integrated dataset for downstream notebooks
print("Saving integrated dataset...")

# Create processed data directory if it doesn't exist
processed_dir = Path("../../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

# Save the integrated dataset
output_file = processed_dir / "integrated_dataset.csv"
merged_df.to_csv(output_file, index=False)

print(f"Integrated dataset saved to: {output_file}")
print(f"Dataset shape: {merged_df.shape}")
print("Ready for downstream analysis and modeling.")

Saving integrated dataset...
Integrated dataset saved to: ..\..\data\processed\integrated_dataset.csv
Dataset shape: (3976135, 17)
Ready for downstream analysis and modeling.


In [22]:
# 5. Weather-Outage Correlation Analysis

print("\n" + "="*50)
print("WEATHER-OUTAGE CORRELATION ANALYSIS")
print("="*50)

# Calculate correlations with target variable
weather_vars = ['tmax', 'tmin', 'prcp']
correlations = {}

for var in weather_vars:
    corr = merged_df[var].corr(merged_df['outage_occurred'])
    correlations[var] = corr
    print(f"{var.upper()} correlation with outages: {corr:.4f}")

# Weather variable distributions by outage status
print("\nWeather Distributions by Outage Status:")
weather_comparison = merged_df.groupby('outage_occurred')[weather_vars].agg(['mean', 'std']).round(2)
display(weather_comparison)

# Extreme weather analysis
print("\nExtreme Weather Event Analysis:")

# Define extreme thresholds based on percentiles
extreme_thresholds = {
    'tmax_extreme': merged_df['tmax'].quantile(0.95),
    'tmin_extreme': merged_df['tmin'].quantile(0.05),
    'prcp_extreme': merged_df['prcp'].quantile(0.95)
}

for event, threshold in extreme_thresholds.items():
    var_name = event.split('_')[0]
    if 'min' in event:
        condition = merged_df[var_name] <= threshold
        operator = "≤"
    else:
        condition = merged_df[var_name] >= threshold
        operator = "≥"
    
    extreme_subset = merged_df[condition]
    outage_rate = extreme_subset['outage_occurred'].mean()
    baseline_rate = merged_df['outage_occurred'].mean()
    
    print(f"{var_name.upper()} {operator} {threshold:.1f}: {outage_rate:.1%} outage rate "
          f"({len(extreme_subset):,} observations, {outage_rate/baseline_rate:.1f}x baseline)")

# Combined weather stress analysis
print("\nCombined Weather Stress Events:")
# High temp + high precip
hot_wet = (merged_df['tmax'] >= merged_df['tmax'].quantile(0.8)) & (merged_df['prcp'] >= merged_df['prcp'].quantile(0.8))
hot_wet_rate = merged_df[hot_wet]['outage_occurred'].mean()
print(f"High temp + high precip: {hot_wet_rate:.1%} outage rate ({hot_wet.sum():,} observations)")

# Cold + precip
cold_wet = (merged_df['tmin'] <= merged_df['tmin'].quantile(0.2)) & (merged_df['prcp'] >= merged_df['prcp'].quantile(0.8))
cold_wet_rate = merged_df[cold_wet]['outage_occurred'].mean()
print(f"Cold + high precip: {cold_wet_rate:.1%} outage rate ({cold_wet.sum():,} observations)")


WEATHER-OUTAGE CORRELATION ANALYSIS
TMAX correlation with outages: 0.0077
TMIN correlation with outages: 0.0042
PRCP correlation with outages: -0.0004

Weather Distributions by Outage Status:


tmax         tmin         prcp      
                  mean    std  mean    std  mean   std
outage_occurred                                       
0                17.90  11.62  5.98  10.92  2.94  7.71
1                18.08  11.54  6.07  10.57  2.93  7.60


Extreme Weather Event Analysis:
TMAX ≥ 33.2: 52.9% outage rate (199,380 observations, 1.1x baseline)
TMIN ≤ -12.9: 44.8% outage rate (198,857 observations, 0.9x baseline)
PRCP ≥ 16.5: 50.3% outage rate (198,981 observations, 1.0x baseline)

Combined Weather Stress Events:
High temp + high precip: 50.4% outage rate (168,250 observations)
Cold + high precip: 49.0% outage rate (73,985 observations)


In [23]:
# Descriptive Statistics
import pandas as pd

print("=" * 60)
print("DESCRIPTIVE STATISTICS")
print("=" * 60)

# Create comprehensive statistics table
key_vars = ['tmax', 'tmin', 'prcp', 'customers_out', 'outage_occurred']

# Calculate all statistics
stats_data = []
for var in key_vars:
    data = merged_df[var]
    stats_data.append({
        'Variable': var,
        'Count': f"{data.count():,}",
        'Mean': f"{data.mean():.3f}",
        'Median': f"{data.median():.3f}",
        'Std Dev': f"{data.std():.3f}",
        'Variance': f"{data.var():.3f}",
        'Min': f"{data.min():.3f}",
        'Max': f"{data.max():.3f}",
        'Skewness': f"{data.skew():.3f}",
        'Kurtosis': f"{data.kurtosis():.3f}"
    })

stats_df = pd.DataFrame(stats_data)
print("\nMAIN STATISTICS TABLE:")
display(stats_df)

# Create percentiles table
print("\nPERCENTILES TABLE:")
percentiles_data = []
percentiles = [5, 25, 50, 75, 95]
for var in key_vars:
    data = merged_df[var]
    row = {'Variable': var}
    for p in percentiles:
        row[f'{p}th'] = f"{data.quantile(p/100):.1f}"
    percentiles_data.append(row)

percentiles_df = pd.DataFrame(percentiles_data)
display(percentiles_df)

# Variable-specific insights table
print("\nVARIABLE-SPECIFIC INSIGHTS:")

# Temperature analysis
temp_range = merged_df['tmax'] - merged_df['tmin']
temp_stats = pd.DataFrame([
    ['Daily temp range (mean)', f"{temp_range.mean():.1f}°C"],
    ['Daily temp range (std)', f"{temp_range.std():.1f}°C"],
    ['Extreme cold days (< -20°C)', f"{(merged_df['tmin'] < -20).sum():,}"],
    ['Extreme hot days (> 40°C)', f"{(merged_df['tmax'] > 40).sum():,}"]
], columns=['Metric', 'Value'])

print("\nTemperature Analysis:")
display(temp_stats)

# Precipitation analysis
prcp_nonzero = merged_df[merged_df['prcp'] > 0]['prcp']
precip_stats = pd.DataFrame([
    ['Days with precipitation', f"{len(prcp_nonzero):,} ({len(prcp_nonzero)/len(merged_df):.1%})"],
    ['Mean on rainy days', f"{prcp_nonzero.mean():.1f}mm"],
    ['Std on rainy days', f"{prcp_nonzero.std():.1f}mm"],
    ['Heavy rain days (>25mm)', f"{(merged_df['prcp'] > 25).sum():,}"],
    ['Extreme rain days (>75mm)', f"{(merged_df['prcp'] > 75).sum():,}"]
], columns=['Metric', 'Value'])

print("\nPrecipitation Analysis:")
display(precip_stats)

# Customer impact analysis
customers_nonzero = merged_df[merged_df['customers_out'] > 0]['customers_out']
customer_stats = pd.DataFrame([
    ['Events with customer data', f"{len(customers_nonzero):,}"],
    ['Mean customers affected', f"{customers_nonzero.mean():.0f}"],
    ['Std customers affected', f"{customers_nonzero.std():.0f}"],
    ['Coefficient of variation', f"{customers_nonzero.std()/customers_nonzero.mean():.2f}"],
    ['Large outages (>1000)', f"{(customers_nonzero > 1000).sum():,}"],
    ['Major outages (>10000)', f"{(customers_nonzero > 10000).sum():,}"]
], columns=['Metric', 'Value'])

print("\nCustomer Impact Analysis:")
display(customer_stats)

DESCRIPTIVE STATISTICS

MAIN STATISTICS TABLE:


,Variable,Count,Mean,Median,Std Dev,Variance,Min,Max,Skewness,Kurtosis
0,tmax,"3,976,135",17.989,20.040,11.577,134.023,-30.860,48.570,-0.569,-0.435
1,tmin,"3,976,135",6.021,6.650,10.747,115.508,-42.620,32.110,-0.464,-0.191
2,prcp,"3,976,135",2.933,0.000,7.656,58.612,0.000,330.800,5.622,57.841
3,customers_out,"3,976,135",41.716,0.000,1297.208,1682747.766,0.000,709360.000,191.883,63261.749
4,outage_occurred,"3,976,135",0.499,0.000,0.500,0.250,0.000,1.000,0.004,-2.000



PERCENTILES TABLE:


,Variable,5th,25th,50th,75th,95th
0,tmax,-2.7,9.6,20.0,27.6,33.2
1,tmin,-12.9,-1.3,6.7,14.7,21.6
2,prcp,0.0,0.0,0.0,2.0,16.5
3,customers_out,0.0,0.0,0.0,2.0,53.0
4,outage_occurred,0.0,0.0,0.0,1.0,1.0



VARIABLE-SPECIFIC INSIGHTS:

Temperature Analysis:


,Metric,Value
0,Daily temp range (mean),12.0°C
1,Daily temp range (std),4.1°C
2,Extreme cold days (< -20°C),"65,809"
3,Extreme hot days (> 40°C),"1,465"



Precipitation Analysis:


,Metric,Value
0,Days with precipitation,"1,760,266 (44.3%)"
1,Mean on rainy days,6.6mm
2,Std on rainy days,10.4mm
3,Heavy rain days (>25mm),"96,292"
4,Extreme rain days (>75mm),"3,813"



Customer Impact Analysis:


,Metric,Value
0,Events with customer data,"1,774,433"
1,Mean customers affected,93
2,Std customers affected,1941
3,Coefficient of variation,20.76
4,Large outages (>1000),"24,349"
5,Major outages (>10000),"1,784"
